Adınız: Hilal Sevda Soyadınız: Akatekin Okul Numaranız: 2212721042 GitHub Repo Bağlantısı: https://github.com/hillsvda/YoloV8_Nesne_Tespiti-

In [ ]:
# Tüm kütüphaneleri, sınıfı ve başlatma bloğunu içeren birleşik kod.

import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout, QPushButton, QLabel, QFileDialog
from PyQt5.QtCore import Qt, QSize
from PyQt5.QtGui import QPixmap, QImage
import cv2
from ultralytics import YOLO
import os
from PIL import Image
from roboflow import Roboflow
import shutil

# --- MODEL YÜKLEME ---
# Eğitimi tamamlandığında best.pt dosyasının yerel yolu.
MODEL_PATH = "C:\\Users\\huaweı\\Desktop\\Notebooks\\runs\\detect\\yolov8_final_training\\weights\\best.pt" 
# NOT: Eğer bu yol yanlışsa, uygulamanız Model Yükleme Hatası verecektir.

# --- ROBODLOW VERİSİ VE API BİLGİLERİ ---
ROBOFLOW_API_KEY = "PYcXKCuxNBLpewFDcgiq"
WORKSPACE_NAME = "projee-krzxk"
PROJECT_NAME = "fareyibul3-kornh"

# 1. Veri Seti İndirme ve Eğitim
try:
    print("Roboflow Veri Seti İndiriliyor...")
    rf = Roboflow(api_key=ROBOFLOW_API_KEY)
    project = rf.workspace(WORKSPACE_NAME).project(PROJECT_NAME)
    version = project.version(1)
    dataset = version.download("yolov8") # Veri setini indirir
    
    # 2. YOLOv8 Model Eğitimi (Kısa Epoklu Deneme)
    print("\nYOLOv8 Model Eğitimi Başlatılıyor...")
    model_yolo = YOLO('yolov8n.pt')
    DATA_YAML_PATH = os.path.join(dataset.location, 'data.yaml')
    
    # DİKKAT: Proje için 50 epok gerekir, burada HIZLI TEST için 5 epok kullanıyoruz.
    results = model_yolo.train(data=DATA_YAML_PATH, epochs=50, imgsz=640, name='yolov8_final_training') 
    
    print("\nEğitim Tamamlandı. best.pt model dosyası oluşturuldu.")
    
except Exception as e:
    print(f"HATA: Veri setini indirirken veya eğitirken sorun oluştu. Hata: {e}")
    # Eğer eğitim başarısız olursa, uygulamanın çökmesini önlemek için modeli geçici olarak yükleyelim
    model_yolo = None


# --- PYQT5 SINIF TANIMI (YOLO TESPİT UYGULAMASI) ---
class YoloDetectorApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("YOLOv8 Nesne Tespiti ve PyQt5 Arayüzü")
        self.setGeometry(100, 100, 1000, 600)
        
        # 1. Modeli Yükleme (Eğitilmiş best.pt dosyasını kullanır)
        try:
            self.model = YOLO(MODEL_PATH)
        except Exception as e:
            # Model bulunamazsa veya yüklenemezse hata verir (DLL/Path sorunu)
            print(f"HATA: Model yüklenemedi. Yolu kontrol edin: {MODEL_PATH}. Hata: {e}")
            self.model = None

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.main_layout = QVBoxLayout(self.central_widget)

        # 2. Görüntü Panelleri
        self.image_display_area = QHBoxLayout()
        self.original_label = QLabel("Original Image")
        self.original_label.setAlignment(Qt.AlignCenter)
        self.original_label.setStyleSheet("border: 1px solid gray; background-color: #f0f0f0;")
        self.tagged_label = QLabel("Tagged Image")
        self.tagged_label.setAlignment(Qt.AlignCenter)
        self.tagged_label.setStyleSheet("border: 1px solid gray; background-color: #f0f0f0;")

        self.image_display_area.addWidget(self.original_label)
        self.image_display_area.addWidget(self.tagged_label)
        self.main_layout.addLayout(self.image_display_area)

        # 3. Sonuç Alanı
        self.results_label = QLabel("Tespit Edilen Nesneler: 0")
        self.main_layout.addWidget(self.results_label)

        # 4. Butonlar
        self.button_layout = QHBoxLayout()
        self.btn_select = QPushButton("Select Image")
        self.btn_select.clicked.connect(self.select_image)
        self.btn_test = QPushButton("Test Image (Nesne Tespiti)")
        self.btn_test.clicked.connect(self.run_detection)
        self.btn_save = QPushButton("Save Image")
        self.btn_save.clicked.connect(self.save_image)
        
        self.button_layout.addWidget(self.btn_select)
        self.button_layout.addWidget(self.btn_test)
        self.button_layout.addWidget(self.btn_save)
        self.main_layout.addLayout(self.button_layout)
        
        self.current_image_path = None
        self.detected_image_cv = None 

    def select_image(self):
        from PyQt5.QtWidgets import QFileDialog
        fname, _ = QFileDialog.getOpenFileName(self, 'Görüntü Seç', '', "Image files (*.jpg *.png *.jpeg)")
        if fname:
            self.current_image_path = fname
            self.display_image(self.original_label, fname)
            self.tagged_label.clear()
            self.tagged_label.setText("Tagged Image")
            self.results_label.setText("Tespit Edilen Nesneler: 0")

    def display_image(self, label_widget, path):
        pixmap = QPixmap(path)
        if not pixmap.isNull():
            label_widget.setPixmap(pixmap.scaled(label_widget.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
            label_widget.setText("")

    def run_detection(self):
        if not self.current_image_path or not self.model:
            print("Lütfen bir görüntü seçin veya modelin yüklendiğinden emin olun.")
            return
        
        results = self.model(self.current_image_path, verbose=False)
        
        if results and len(results) > 0:
            r = results[0]
            names = self.model.names
            detected_counts = {}
            
            im_array = r.plot()
            self.detected_image_cv = im_array
            
            im_array_rgb = cv2.cvtColor(im_array, cv2.COLOR_BGR2RGB) 
            height, width, channel = im_array_rgb.shape
            bytes_per_line = 3 * width
            q_img = QImage(im_array_rgb.data, width, height, bytes_per_line, QImage.Format_RGB888)
            
            self.tagged_label.setPixmap(QPixmap.fromImage(q_img).scaled(self.tagged_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation))
            self.tagged_label.setText("")

            if r.boxes and r.boxes.cls is not None:
                for c in r.boxes.cls:
                    class_name = names[int(c)]
                    detected_counts[class_name] = detected_counts.get(class_name, 0) + 1
            
            count_text = ", ".join([f"{name}: {count}" for name, count in detected_counts.items()])
            total_detected = len(r.boxes.cls) if r.boxes.cls is not None else 0
            self.results_label.setText(f"Tespit Edilen Nesneler ({total_detected} adet): {count_text}")
        else:
            self.tagged_label.clear()
            self.tagged_label.setText("Tespit Yok")
            self.results_label.setText("Tespit Edilen Nesneler: 0")

    def save_image(self):
        from PyQt5.QtWidgets import QFileDialog
        
        if self.detected_image_cv is None:
            print("Kaydedilecek tespit edilmiş görüntü yok.")
            return

        fname, _ = QFileDialog.getSaveFileName(self, "Tespit Edilen Görüntüyü Kaydet", "", "JPG Files (*.jpg);;PNG Files (*.png)")
        
        if fname:
            cv2.imwrite(fname, self.detected_image_cv)
            print(f"Görüntü kaydedildi: {fname}")


# --- Uygulamayı Başlatma Bloğu ---
app = QApplication(sys.argv)
window = YoloDetectorApp()
window.show()
sys.exit(app.exec_())

Roboflow Veri Seti İndiriliyor...
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Fareyibul3-1 in yolov8:: 100%|███████████████████| 456/456 [00:00<00:00, 1542.75it/s]



YOLOv8 Model Eğitimi Başlatılıyor...
New https://pypi.org/project/ultralytics/8.3.232 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.230  Python-3.11.4 torch-2.9.1+cpu CPU (11th Gen Intel Core i5-1135G7 @ 2.40GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\huawe\Desktop\Notebooks\Fareyibul3-1\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.